# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), April 27, 2019**

In [1]:
using DataFrames
using BenchmarkTools

## Performance tips

### Access by column number is faster than by name

In [2]:
x = DataFrame(rand(5, 1000))
@btime $x[500];
@btime $x.x500;

  4.112 ns (0 allocations: 0 bytes)
  13.879 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [3]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y, z = x[1], x[2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  83.589 ms (5999023 allocations: 122.06 MiB)


In [4]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    f_inner(x[1], x[2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    dot(x[1], x[2])
end

@btime f_barrier();
@btime f_inbuilt();

  12.459 ms (45 allocations: 30.52 MiB)
  12.350 ms (45 allocations: 30.52 MiB)


In [5]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y::Vector{Float64}, z::Vector{Float64} = x[1], x[2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  12.330 ms (45 allocations: 30.52 MiB)


### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between `DataFrame` and `DataFrame!` (copying vs non-copying data frame creation)

In [6]:
function f1()
    x = DataFrame!([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame!(x) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  20.568 ms (1949837 allocations: 37.42 MiB)
  23.942 ms (1950037 allocations: 45.06 MiB)
  1.632 ms (937 allocations: 7.69 MiB)
  2.739 ms (1137 allocations: 15.32 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [7]:
x = DataFrame(rand(10^6, 5))
y = DataFrame(transpose(1.0:5.0))
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5)) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  14.224 ms (175 allocations: 38.16 MiB)
  172.509 ns (1 allocation: 16 bytes)
  199.227 ns (5 allocations: 80 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [8]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.660 ms (7 allocations: 7.63 MiB)
 categorical:
  18.671 ms (4 allocations: 608 bytes)
String
 raw:
  23.233 ms (4 allocations: 608 bytes)
 categorical:
  28.929 ms (4 allocations: 608 bytes)
Union{Missing, Int64}
 raw:
  10.270 ms (4 allocations: 624 bytes)
 categorical:
  18.987 ms (4 allocations: 608 bytes)
Union{Missing, String}
 raw:
  19.391 ms (4 allocations: 608 bytes)
 categorical:
  30.102 ms (4 allocations: 608 bytes)


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [9]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [10]:
@btime by($df, :x, v -> sum(v.y)) # traditional syntax, slow

  353.072 ms (192 allocations: 471.33 MiB)


,x,x1
,Char,Int64
1,'b',2498764
2,'d',2499255
3,'a',2500996
4,'c',2500985


In [11]:
@btime by($df, :x, :y=>sum) # use column selector

  216.236 ms (152 allocations: 395.04 MiB)


,x,y_sum
,Char,Int64
1,'b',2498764
2,'d',2499255
3,'a',2500996
4,'c',2500985


In [12]:
categorical!(df, :x);

In [13]:
@btime by($df, :x, :y=>sum)

  110.713 ms (187 allocations: 190.75 MiB)


,x,y_sum
,Categorical…,Int64
1,'a',2500996
2,'b',2498764
3,'c',2500985
4,'d',2499255


In [14]:
using PooledArrays

In [15]:
df.x = PooledArray{Char}(df.x)

10000000-element PooledArray{Char,UInt8,1,Array{UInt8,1}}:
 'b'
 'd'
 'a'
 'd'
 'b'
 'c'
 'a'
 'd'
 'c'
 'a'
 'c'
 'c'
 'a'
 ⋮  
 'b'
 'b'
 'b'
 'b'
 'd'
 'c'
 'a'
 'a'
 'b'
 'b'
 'c'
 'c'

In [16]:
@btime by($df, :x, :y=>sum)

  128.121 ms (167 allocations: 162.14 MiB)


,x,y_sum
,Char,Int64
1,'b',2498764
2,'d',2499255
3,'a',2500996
4,'c',2500985


### Use views instead of materializing a new DataFrame

In [17]:
x = DataFrame(rand(100, 1000))

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058
2,0.774187,0.356867,0.885526,0.11275,0.780669,0.593082,0.939199
3,0.609714,0.288531,0.355283,0.480804,0.593935,0.893138,0.99899
4,0.694783,0.570324,0.0789985,0.818004,0.554942,0.0996372,0.861696
5,0.791302,0.905282,0.117797,0.406855,0.681548,0.819496,0.0545339
6,0.0740549,0.679826,0.168822,0.518838,0.335502,0.565356,0.819869
7,0.621834,0.759774,0.562538,0.560436,0.269123,0.340253,0.532622
8,0.1039,0.919697,0.418945,0.813356,0.716439,0.545716,0.783745
9,0.954089,0.808411,0.0782112,0.154967,0.730512,0.57385,0.0323516


In [18]:
@btime $x[1:1, :]

  143.422 μs (1511 allocations: 194.41 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865


In [19]:
@btime $x[1, :]

  22.618 ns (1 allocation: 32 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865


In [20]:
@btime view($x, 1:1, :)

  23.132 ns (1 allocation: 48 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865


In [21]:
@btime $x[1:1, 1:20]

  4.847 μs (55 allocations: 7.23 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865


In [22]:
@btime $x[1, 1:20]

  26.216 ns (2 allocations: 80 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865


In [23]:
@btime view($x, 1:1, 1:20)

  26.837 ns (2 allocations: 96 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.469427,0.893605,0.348301,0.189853,0.602694,0.427064,0.120058,0.403865
